In [ ]:
"""
COMPLETE INTEGRATED ANALYSIS
============================
Runs ALL analysis including ML models and deep insights

Author: Abhinav Anand
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (classification_report, confusion_matrix, roc_auc_score,
                            roc_curve, accuracy_score, precision_score, recall_score,
                            f1_score, matthews_corrcoef)
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("🚀 COMPLETE INTEGRATED TRADER BEHAVIOR ANALYSIS")
print("="*80)
print(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

# ============================================================================
# PART 1: DATA LOADING
# ============================================================================

print("="*80)
print("PART 1: DATA LOADING")
print("="*80)

fear_greed = pd.read_csv('fear_greed_index.csv')
trades = pd.read_csv('historical_data.csv')

print(f"✅ Fear & Greed: {fear_greed.shape}")
print(f"✅ Trading Data: {trades.shape}")

# Clean data
fear_greed['date'] = pd.to_datetime(fear_greed['date'], format='mixed', dayfirst=True)
trades['Timestamp IST'] = pd.to_datetime(trades['Timestamp IST'], format='mixed', dayfirst=True)
trades['date'] = pd.to_datetime(trades['Timestamp IST'].dt.date)
trades = trades.dropna(subset=['Closed PnL', 'Size USD', 'Execution Price'])

# Create features
trades['profitable'] = trades['Closed PnL'] > 0
trades['hour'] = trades['Timestamp IST'].dt.hour
trades['day_of_week'] = trades['Timestamp IST'].dt.day_name()

# Merge
merged_df = trades.merge(
    fear_greed[['date', 'value', 'classification']],
    on='date',
    how='left'
)
merged_df = merged_df.rename(columns={
    'value': 'fear_greed_score',
    'classification': 'market_sentiment'
})

print(f"✅ Merged: {len(merged_df):,} trades\n")

# ============================================================================
# PART 2: FEATURE ENGINEERING FOR ML
# ============================================================================

print("="*80)
print("PART 2: FEATURE ENGINEERING FOR ML")
print("="*80)

# Create ML features
df_ml = merged_df.copy()
df_ml['day_of_week_num'] = df_ml['Timestamp IST'].dt.dayofweek
df_ml['is_weekend'] = (df_ml['day_of_week_num'] >= 5).astype(int)
df_ml['month'] = df_ml['Timestamp IST'].dt.month

# Sentiment features
df_ml['sentiment_extreme'] = ((df_ml['fear_greed_score'] < 25) |
                              (df_ml['fear_greed_score'] > 75)).astype(int)
df_ml['high_greed'] = (df_ml['fear_greed_score'] > 70).astype(int)
df_ml['high_fear'] = (df_ml['fear_greed_score'] < 30).astype(int)

# Trade features
df_ml['log_trade_size'] = np.log1p(df_ml['Size USD'])
df_ml['log_price'] = np.log1p(df_ml['Execution Price'])
df_ml['is_buy'] = (df_ml['Side'] == 'Buy').astype(int)
df_ml['is_crossed'] = df_ml['Crossed'].astype(int)

# Encode coin
le_coin = LabelEncoder()
df_ml['coin_encoded'] = le_coin.fit_transform(df_ml['Coin'])

# Trader historical features
df_ml = df_ml.sort_values(['Account', 'Timestamp IST'])
df_ml['trader_trade_count'] = df_ml.groupby('Account').cumcount()
df_ml['trader_cumulative_pnl'] = df_ml.groupby('Account')['Closed PnL'].cumsum().shift(1).fillna(0)
df_ml['trader_recent_winrate'] = (df_ml.groupby('Account')['profitable']
                                  .rolling(window=10, min_periods=1)
                                  .mean()
                                  .reset_index(level=0, drop=True))

print(f"✅ Created {len(df_ml.columns)} features\n")

# ============================================================================
# PART 3: MACHINE LEARNING MODELS
# ============================================================================

print("="*80)
print("PART 3: TRAINING MACHINE LEARNING MODELS")
print("="*80)

# Select features
feature_cols = [
    'fear_greed_score', 'sentiment_extreme', 'high_greed', 'high_fear',
    'log_trade_size', 'log_price', 'hour', 'day_of_week_num', 'is_weekend',
    'month', 'is_buy', 'is_crossed', 'coin_encoded',
    'trader_trade_count', 'trader_cumulative_pnl', 'trader_recent_winrate'
]

# Prepare data
df_clean = df_ml[feature_cols + ['profitable']].dropna()
X = df_clean[feature_cols]
y = df_clean['profitable'].astype(int)

print(f"\n📊 ML Dataset: {len(X):,} samples, {len(feature_cols)} features")
print(f"   Class balance: {y.mean()*100:.1f}% profitable\n")

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training: {len(X_train):,} | Testing: {len(X_test):,}\n")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train models
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced'),
    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=10,
                                           random_state=42, n_jobs=-1, class_weight='balanced'),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, max_depth=5,
                                                   learning_rate=0.1, random_state=42)
}

results = {}

for name, model in models.items():
    print(f"{'='*60}")
    print(f"🤖 TRAINING: {name}")
    print('='*60)

    # Train
    if name == 'Logistic Regression':
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)
    mcc = matthews_corrcoef(y_test, y_pred)

    print(f"\n📊 TEST SET PERFORMANCE:")
    print(f"   Accuracy:  {accuracy:.4f}")
    print(f"   Precision: {precision:.4f}")
    print(f"   Recall:    {recall:.4f}")
    print(f"   F1-Score:  {f1:.4f}")
    print(f"   AUC-ROC:   {auc:.4f}")
    print(f"   MCC:       {mcc:.4f}")

    # Cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    if name == 'Logistic Regression':
        cv_scores = cross_val_score(model, X_train_scaled, y_train,
                                   cv=cv, scoring='roc_auc', n_jobs=-1)
    else:
        cv_scores = cross_val_score(model, X_train, y_train,
                                   cv=cv, scoring='roc_auc', n_jobs=-1)

    print(f"\n🔄 5-FOLD CROSS-VALIDATION:")
    print(f"   Mean AUC: {cv_scores.mean():.4f} (+/- {cv_scores.std()*2:.4f})")

    print(f"\n📋 CLASSIFICATION REPORT:")
    print(classification_report(y_test, y_pred,
                               target_names=['Unprofitable', 'Profitable'],
                               digits=4))

    results[name] = {
        'model': model,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc,
        'mcc': mcc,
        'cv_scores': cv_scores,
        'y_test': y_test,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }

# ============================================================================
# PART 4: FEATURE IMPORTANCE
# ============================================================================

print("\n" + "="*80)
print("PART 4: FEATURE IMPORTANCE ANALYSIS")
print("="*80)

rf_model = results['Random Forest']['model']
importances = rf_model.feature_importances_

feature_importance_df = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': importances,
    'Importance_Pct': importances / importances.sum() * 100
}).sort_values('Importance', ascending=False)

print("\n🎯 TOP 10 MOST IMPORTANT FEATURES:")
print(feature_importance_df.head(10).to_string(index=False))

top_3_pct = feature_importance_df.head(3)['Importance_Pct'].sum()
print(f"\n💡 Top 3 features account for {top_3_pct:.1f}% of predictive power")

# ============================================================================
# PART 5: ADVANCED STATISTICAL TESTS
# ============================================================================

print("\n" + "="*80)
print("PART 5: ADVANCED STATISTICAL VALIDATION")
print("="*80)

# Correlation test
from scipy.stats import pearsonr, spearmanr
pearson_corr, pearson_p = pearsonr(
    merged_df['fear_greed_score'].dropna(),
    merged_df.loc[merged_df['fear_greed_score'].notna(), 'Closed PnL']
)

print(f"\n📊 PEARSON CORRELATION (Sentiment vs PnL):")
print(f"   Correlation: {pearson_corr:.4f}")
print(f"   P-value: {pearson_p:.6f}")
print(f"   Result: {'✅ SIGNIFICANT' if pearson_p < 0.05 else '❌ Not significant'}")

# ANOVA test
sentiment_groups = [
    merged_df[merged_df['market_sentiment'] == s]['Closed PnL'].values
    for s in merged_df['market_sentiment'].unique()
    if len(merged_df[merged_df['market_sentiment'] == s]) > 0
]

f_stat, anova_p = stats.f_oneway(*sentiment_groups)

print(f"\n📊 ONE-WAY ANOVA (PnL across sentiments):")
print(f"   F-statistic: {f_stat:.4f}")
print(f"   P-value: {anova_p:.6f}")
print(f"   Result: {'✅ SIGNIFICANT difference' if anova_p < 0.05 else '❌ No significant difference'}")

# ============================================================================
# PART 6: TRADER CLUSTERING
# ============================================================================

print("\n" + "="*80)
print("PART 6: TRADER CLUSTERING ANALYSIS")
print("="*80)

trader_profiles = merged_df.groupby('Account').agg({
    'Closed PnL': ['mean', 'std', 'sum'],
    'Size USD': 'mean',
    'profitable': 'mean',
    'fear_greed_score': 'mean',
    'Timestamp IST': 'count'
}).reset_index()

trader_profiles.columns = ['Account', 'Avg_PnL', 'PnL_Volatility', 'Total_PnL',
                           'Avg_Trade_Size', 'Win_Rate', 'Avg_Sentiment', 'Trade_Count']

active_traders = trader_profiles[trader_profiles['Trade_Count'] >= 10].copy()

# K-means clustering
cluster_features = ['Avg_PnL', 'PnL_Volatility', 'Avg_Trade_Size',
                   'Win_Rate', 'Avg_Sentiment', 'Trade_Count']

X_cluster = active_traders[cluster_features].fillna(0)
scaler_cluster = StandardScaler()
X_cluster_scaled = scaler_cluster.fit_transform(X_cluster)

n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
active_traders['Cluster'] = kmeans.fit_predict(X_cluster_scaled)

print(f"\n👥 Identified {n_clusters} trader clusters from {len(active_traders)} active traders\n")

for i in range(n_clusters):
    cluster_data = active_traders[active_traders['Cluster'] == i]
    print(f"CLUSTER {i}: {len(cluster_data)} traders ({len(cluster_data)/len(active_traders)*100:.1f}%)")
    print(f"   Avg PnL: ${cluster_data['Avg_PnL'].mean():,.2f}")
    print(f"   Win Rate: {cluster_data['Win_Rate'].mean()*100:.1f}%")
    print(f"   Avg Trade Size: ${cluster_data['Avg_Trade_Size'].mean():,.2f}")
    print(f"   Avg Sentiment: {cluster_data['Avg_Sentiment'].mean():.1f}\n")

# ============================================================================
# PART 7: COMPREHENSIVE VISUALIZATIONS
# ============================================================================

print("="*80)
print("PART 7: GENERATING VISUALIZATIONS")
print("="*80)

# ML Performance Dashboard
fig = plt.figure(figsize=(18, 12))

# 1. Model Accuracy Comparison
ax1 = plt.subplot(2, 3, 1)
model_names = list(results.keys())
accuracies = [results[m]['accuracy'] for m in model_names]
bars = ax1.bar(range(len(model_names)), accuracies,
              color=['steelblue', 'coral', 'lightgreen'],
              edgecolor='black', linewidth=2)
ax1.set_xticks(range(len(model_names)))
ax1.set_xticklabels(model_names, rotation=15, ha='right', fontsize=9)
ax1.set_ylabel('Accuracy', fontweight='bold')
ax1.set_title('Model Accuracy', fontsize=12, fontweight='bold')
ax1.set_ylim([0.3, 0.7])
ax1.grid(axis='y', alpha=0.3)
for i, (bar, v) in enumerate(zip(bars, accuracies)):
    ax1.text(i, v + 0.01, f'{v:.3f}', ha='center', fontweight='bold', fontsize=9)

# 2. AUC-ROC Comparison
ax2 = plt.subplot(2, 3, 2)
aucs = [results[m]['auc'] for m in model_names]
bars = ax2.bar(range(len(model_names)), aucs,
              color=['steelblue', 'coral', 'lightgreen'],
              edgecolor='black', linewidth=2)
ax2.set_xticks(range(len(model_names)))
ax2.set_xticklabels(model_names, rotation=15, ha='right', fontsize=9)
ax2.set_ylabel('AUC-ROC', fontweight='bold')
ax2.set_title('Model AUC-ROC', fontsize=12, fontweight='bold')
ax2.set_ylim([0.3, 0.7])
ax2.grid(axis='y', alpha=0.3)
for i, (bar, v) in enumerate(zip(bars, aucs)):
    ax2.text(i, v + 0.01, f'{v:.3f}', ha='center', fontweight='bold', fontsize=9)

# 3. ROC Curves
ax3 = plt.subplot(2, 3, 3)
for name in model_names:
    fpr, tpr, _ = roc_curve(results[name]['y_test'], results[name]['y_pred_proba'])
    ax3.plot(fpr, tpr, label=f"{name}\n(AUC={results[name]['auc']:.3f})", linewidth=2)
ax3.plot([0, 1], [0, 1], 'k--', label='Random\n(AUC=0.5)', linewidth=1.5)
ax3.set_xlabel('False Positive Rate', fontweight='bold')
ax3.set_ylabel('True Positive Rate', fontweight='bold')
ax3.set_title('ROC Curves', fontsize=12, fontweight='bold')
ax3.legend(loc='lower right', fontsize=8)
ax3.grid(alpha=0.3)

# 4. Feature Importance
ax4 = plt.subplot(2, 3, 4)
top_10 = feature_importance_df.head(10)
ax4.barh(range(len(top_10)), top_10['Importance'].values,
        color='purple', edgecolor='black', linewidth=1)
ax4.set_yticks(range(len(top_10)))
ax4.set_yticklabels(top_10['Feature'].values, fontsize=9)
ax4.set_xlabel('Importance', fontweight='bold')
ax4.set_title('Top 10 Features', fontsize=12, fontweight='bold')
ax4.invert_yaxis()
ax4.grid(axis='x', alpha=0.3)

# 5. Confusion Matrix (Best Model)
ax5 = plt.subplot(2, 3, 5)
best_model = max(results.keys(), key=lambda k: results[k]['auc'])
cm = confusion_matrix(results[best_model]['y_test'], results[best_model]['y_pred'])
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax5,
           xticklabels=['Unprofitable', 'Profitable'],
           yticklabels=['Unprofitable', 'Profitable'])
ax5.set_title(f'Confusion Matrix\n({best_model})', fontsize=12, fontweight='bold')
ax5.set_ylabel('True', fontweight='bold')
ax5.set_xlabel('Predicted', fontweight='bold')

# 6. All Metrics Comparison
ax6 = plt.subplot(2, 3, 6)
metrics = ['Accuracy', 'Precision', 'Recall', 'F1', 'AUC']
x = np.arange(len(metrics))
width = 0.25

colors_bar = ['steelblue', 'coral', 'lightgreen']
for i, (name, color) in enumerate(zip(model_names, colors_bar)):
    values = [
        results[name]['accuracy'],
        results[name]['precision'],
        results[name]['recall'],
        results[name]['f1'],
        results[name]['auc']
    ]
    ax6.bar(x + i*width, values, width, label=name,
           color=color, edgecolor='black', linewidth=1)

ax6.set_ylabel('Score', fontweight='bold')
ax6.set_title('All Metrics', fontsize=12, fontweight='bold')
ax6.set_xticks(x + width)
ax6.set_xticklabels(metrics, fontsize=9)
ax6.legend(fontsize=8)
ax6.grid(axis='y', alpha=0.3)
ax6.set_ylim([0.3, 0.7])

plt.suptitle('🤖 MACHINE LEARNING MODEL PERFORMANCE DASHBOARD',
            fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('ml_performance_complete.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.close()

print("\n✅ ML dashboard saved: 'ml_performance_complete.png'")

# ============================================================================
# FINAL SUMMARY
# ============================================================================

print("\n" + "="*80)
print("🎉 COMPLETE ANALYSIS SUMMARY")
print("="*80)

best_model_name = max(results.keys(), key=lambda k: results[k]['auc'])
best_result = results[best_model_name]

print(f"""
📊 MACHINE LEARNING RESULTS:

Best Model: {best_model_name}
├─ Accuracy:   {best_result['accuracy']:.4f}
├─ Precision:  {best_result['precision']:.4f}
├─ Recall:     {best_result['recall']:.4f}
├─ F1-Score:   {best_result['f1']:.4f}
├─ AUC-ROC:    {best_result['auc']:.4f}
└─ MCC:        {best_result['mcc']:.4f}

Cross-Validation: {best_result['cv_scores'].mean():.4f} (+/- {best_result['cv_scores'].std()*2:.4f})

🎯 TOP 3 PREDICTIVE FEATURES:
{feature_importance_df.head(3)[['Feature', 'Importance_Pct']].to_string(index=False)}

📈 STATISTICAL VALIDATION:
├─ Pearson Correlation: {pearson_corr:.4f} (p={pearson_p:.6f}) {'✅' if pearson_p < 0.05 else '❌'}
└─ ANOVA F-statistic: {f_stat:.2f} (p={anova_p:.6f}) {'✅' if anova_p < 0.05 else '❌'}

👥 TRADER CLUSTERING:
└─ Identified {n_clusters} distinct trader archetypes

📁 GENERATED FILES:
├─ complete_analysis_dashboard.png
├─ timing_heatmaps.png
├─ ml_performance_complete.png
└─ analysis_report.txt

✅ ANALYSIS COMPLETE!
""")

print("="*80)
print(f"Finished: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

🚀 COMPLETE INTEGRATED TRADER BEHAVIOR ANALYSIS
Started: 2025-11-08 08:40:48

PART 1: DATA LOADING
✅ Fear & Greed: (2644, 4)
✅ Trading Data: (211224, 16)
✅ Merged: 211,224 trades

PART 2: FEATURE ENGINEERING FOR ML
✅ Created 36 features

PART 3: TRAINING MACHINE LEARNING MODELS

📊 ML Dataset: 211,218 samples, 16 features
   Class balance: 41.1% profitable

Training: 168,974 | Testing: 42,244

🤖 TRAINING: Logistic Regression

📊 TEST SET PERFORMANCE:
   Accuracy:  0.8859
   Precision: 0.8538
   Recall:    0.8719
   F1-Score:  0.8628
   AUC-ROC:   0.9699
   MCC:       0.7653

🔄 5-FOLD CROSS-VALIDATION:
   Mean AUC: 0.9718 (+/- 0.0004)

📋 CLASSIFICATION REPORT:
              precision    recall  f1-score   support

Unprofitable     0.9092    0.8957    0.9024     24871
  Profitable     0.8538    0.8719    0.8628     17373

    accuracy                         0.8859     42244
   macro avg     0.8815    0.8838    0.8826     42244
weighted avg     0.8864    0.8859    0.8861     42244

🤖 TRAINI